In [3]:
import random
from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw
import os
from PIL import Image
from skimage import io
from skimage import transform as tf
import scipy.misc
import numpy as np
import cv2
import imutils


In [31]:
alphanum = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z','0','1','2','3','4','5','6','7','8','9']
fntSizes = [20,21,22,23,24,25,26,27,28,29,30,31,32]

def createsamples(text_pos,alpha,fnt,seq,fntSize):
    if alpha =='M' or alpha == 'W':
        bgimage = Image.new('RGBA', (fntSize,fntSize), (255, 255, 255, 255))     
    elif alpha =='I':
        bgimage = Image.new('RGBA', (fntSize-10,fntSize), (255, 255, 255, 255))             
    elif alpha in ('0','1','2','3','4','5','6','7','8','9'):
        bgimage = Image.new('RGBA', (fntSize-3,fntSize), (255, 255, 255, 255))             
    else:
        bgimage = Image.new('RGBA', (fntSize-2,fntSize), (255, 255, 255, 255)) 
        
    draw  = ImageDraw.Draw(bgimage)
    draw.text(text_pos,alpha,font=fnt,fill=(0,0,0))
    bgimage.save('Background/samples2/'+alpha+'_'+str(seq)+'.png')
    image = cv2.imread('Background/samples2/'+alpha+'_'+str(seq)+'.png')
    image = imutils.resize(image,height=20)
    white_img = np.zeros((40,40,3), np.uint8)
    white_img[:,:] = (255,255,255)
    #Paste the image onto the white canvas
    x_offset=y_offset=1    
    white_img[y_offset:y_offset+image.shape[0], x_offset:x_offset+image.shape[1]] = image
    cv2.imwrite('Background/samples2/'+alpha+'_'+str(seq)+'.png',white_img)
        
for alpha in alphanum:
    #print(str(seq))
    seq = 1
    for fntSize in fntSizes:
        #for i in range (50):
        text_Pos = (1,1)                
        fnt = ImageFont.truetype('Background/arialbd.ttf', fntSize)
        createsamples(text_Pos,alpha,fnt,seq,fntSize)
        seq = seq +1
                
print ('samples created....')
           


samples created....


In [75]:
for filename in os.listdir('Background/samples2/'):
    img = cv2.imread('Background/samples2/'+filename)
    cv2.imwrite('Background/samplesBW2/'+filename,img)
print('images copied from samples2')

images copied from samples2


In [76]:

for filename in os.listdir('CharSplitTrain/1/'):
    image = cv2.imread('CharSplitTrain/1/'+filename)
    #image = imutils.resize(image,height=20)
    #white_img = np.zeros((40,40,3), np.uint8)
    #white_img[:,:] = (255,255,255)
    #Paste the image onto the white canvas
    #x_offset=y_offset=1    
    #white_img[y_offset:y_offset+image.shape[0], x_offset:x_offset+image.shape[1]] = image
    cv2.imwrite('Background/samplesBW2/'+filename,image)
    #print(filename)
print('images copied from CharSplitTrain')

images copied from CharSplitTrain


In [77]:
def noisy(noise_typ,image):
    
    if noise_typ == "gauss":
        row,col,ch= image.shape
        mean = 0
        var = 0.1
        sigma = var**0.5
        gauss = np.random.normal(mean,sigma,(row,col,ch))
        gauss = gauss.reshape(row,col,ch)
        noisy = image + gauss
        return noisy
    elif noise_typ == "s&p":
        
        row,col,ch = image.shape
        s_vs_p = 0.5
        amount = 0.004
        out = np.copy(image)
        # Salt mode
        num_salt = np.ceil(amount * image.size * s_vs_p)
        coords = [np.random.randint(0, i - 1, int(num_salt))
              for i in image.shape]
        out[coords] = 1

        # Pepper mode
        num_pepper = np.ceil(amount* image.size * (1. - s_vs_p))
        coords = [np.random.randint(0, i - 1, int(num_pepper))
              for i in image.shape]
        out[coords] = 0
        return out
    elif noise_typ == "poisson":
        vals = len(np.unique(image))
        vals = 2 ** np.ceil(np.log2(vals))
        noisy = np.random.poisson(image * vals) / float(vals)
        return noisy
    elif noise_typ =="speckle":
        row,col,ch = image.shape
        gauss = np.random.randn(row,col,ch)
        gauss = gauss.reshape(row,col,ch)        
        noisy = image + image * gauss
        return noisy

for filename in os.listdir('Background/samplesBW2/'):        
    image = cv2.imread('Background/samplesBW2/'+filename)
    #image = noisy('gauss',image)
    #image = noisy('poisson',image)
    image = cv2.blur(image,(2,2))
    ##print(image)
    cv2.imwrite('Background/samplesBW2/'+filename,image)
    #image = noisy(image)
    #print(filename)
    #blendImage(image,'samplesBW/'+filename)
    #cv2.imwrite('samplesBW/'+filename,image)

print('noice induced')



noice induced


In [78]:
print('start reshaping')
alphanum = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z','0','1','2','3','4','5','6','7','8','9']
noOfFiles = len([name for name in os.listdir('Background/samplesBW2/')])
print('number of files'+str(noOfFiles))

reshaped = np.empty([noOfFiles,1600])
Y = np.chararray([noOfFiles,1]) 
j=0
for filename in os.listdir('Background/samplesBW2/'):
    image = io.imread('Background/samplesBW2/'+filename,'0')
    reshaped[j] = image.reshape(1,1600)
    Y[j] = filename[:1]
    j = j+1
    

print('Reshaping Complete')
print('Starting to segregate Training and Validation sets')
Y = np.ravel(Y)
#print(Y)
from sklearn.model_selection import train_test_split
X_Train,X_Test,Y_Train,Y_Test = train_test_split(reshaped,Y)


print('Segregation Completed')

start reshaping
number of files1144
Reshaping Complete
Starting to segregate Training and Validation sets
Segregation Completed


In [79]:
print('starting to feature scaling')

#from sklearn.preprocessing import StandardScaler
#scaler = StandardScaler()
# Fit only to the training data
#scaler.fit(X_Train)
#X_Train = scaler.transform(X_Train)
#X_Test = scaler.transform(X_Test)

print('Feature Scaling Completed')
print('Start training')
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(340,340,340),alpha=0.0001)
mlp.fit(X_Train,Y_Train)


print('Training done...')
print('Now start predicting for the validation set')
predictions = mlp.predict(X_Test)

print('Predictions done')
print('evaluate the results')
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(Y_Test,predictions))

starting to feature scaling
Feature Scaling Completed
Start training
Training done...
Now start predicting for the validation set
Predictions done
evaluate the results
             precision    recall  f1-score   support

       b'0'       1.00      0.75      0.86         8
       b'1'       1.00      0.83      0.91         6
       b'2'       1.00      1.00      1.00         5
       b'3'       1.00      1.00      1.00         2
       b'4'       1.00      1.00      1.00         6
       b'5'       1.00      0.88      0.93         8
       b'6'       0.90      1.00      0.95         9
       b'7'       1.00      1.00      1.00         3
       b'8'       1.00      1.00      1.00         3
       b'9'       1.00      0.78      0.88         9
       b'A'       1.00      0.91      0.95        32
       b'C'       1.00      1.00      1.00         4
       b'D'       1.00      1.00      1.00        13
       b'E'       1.00      1.00      1.00        21
       b'F'       1.00      1.00    

In [80]:
import pickle
pickle.dump(mlp,open('Model output/AlphabetRecognition.sav','wb'))

In [18]:
for filename in os.listdir('Background/Test/'):  
    #BGImage = Image.open('defaultBG.png')
    #img = Image.open('Test/'+filename)    
    #BGImage.paste(img, (15,15))    
    #BGImage.save('TestBW/'+filename)
    
    image = cv2.imread('Background/Test/'+filename)
    gray_masked = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    (thresh, im_bw) = cv2.threshold(gray_masked, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    im_bw = imutils.resize(im_bw,height = 20)
    
    cv2.imwrite('Background/TestBW/'+filename,im_bw)
    #blendImage(image,'TestBW/'+filename)
    
print('Test images created...')

Test images created...


In [19]:
#Paste the BW image onto a standard 50X50 white background
for filename in os.listdir('Background/TestBW/'):
    img = Image.open('Background/TestBW/'+filename)
    bgimage = Image.new('RGBA', (40,40), (255, 255, 255, 255))        
    pos = (1,1)
    bgimage.paste(img,pos)
    bgimage.save('Background/TestBW/'+filename)
    
    image = cv2.imread('Background/TestBW/'+filename)
    image = cv2.blur(image,(5,5))
    cv2.imwrite('Background/TestBW/'+filename,image)
    
print('Test images created...')




Test images created...


In [20]:
print('starting to reshape the test images')
pannumber = ['A','W','S','P','K','5','9','6','7','E']
reshaped1 = np.empty([10,1600])
Y1=np.chararray([len(pannumber),1])
j=0
for alpha in pannumber:
    #for i in range (1,3):
        filename = 'Background/TestBW/'+alpha+'_1.PNG'
        #print(filename)
        image = io.imread(filename,'1')    
        #print(image)
        reshaped1[j] = image.reshape(1,1600)
        Y1[j] = alpha
        j = j+1

Y1 = np.ravel(Y1)

print('reshaping done...')
print('start predicting')
#X1_Test = scaler.transform(reshaped1)
predictions = mlp.predict(reshaped1)
print(predictions)
print(classification_report(Y1,predictions))

starting to reshape the test images
reshaping done...
start predicting
[b'K' b'V' b'G' b'J' b'K' b'9' b'S' b'6' b'7' b'E']
             precision    recall  f1-score   support

       b'5'       0.00      0.00      0.00         1
       b'6'       1.00      1.00      1.00         1
       b'7'       1.00      1.00      1.00         1
       b'9'       0.00      0.00      0.00         1
       b'A'       0.00      0.00      0.00         1
       b'E'       1.00      1.00      1.00         1
       b'G'       0.00      0.00      0.00         0
       b'J'       0.00      0.00      0.00         0
       b'K'       0.50      1.00      0.67         1
       b'P'       0.00      0.00      0.00         1
       b'S'       0.00      0.00      0.00         1
       b'V'       0.00      0.00      0.00         0
       b'W'       0.00      0.00      0.00         1

avg / total       0.35      0.40      0.37        10



/home/kunjithapathams/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/kunjithapathams/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [56]:
print(predictions)

[b'A' b'T' b'M' b'Y' b'M' b'5' b'T' b'E' b'7' b'E']
